In [1]:
import numpy as np
import math

#### Портфель

In [2]:
# задан портфель из двух инструментов
portfolio = [
    {
        'quantity': 1,
        'market_depth': 10_000,
        'execution_lag': 3,
        'ds': False,
        'spot': 100,
        'drift': None,
        'volatility': None
    },
    {
        'quantity': -2,
        'market_depth': 10_000,
        'execution_lag': 1,
        'ds': False,
        'spot': 100,
        'drift': None,
        'volatility': None
    }
]

Предположительно нужно будет завести список каких-то факторов для генерации риск-факторов: цены и волатильность (посоветовали использовать SABR)

In [3]:
# factors

#### Определение периода ликвидации портфеля $H$

In [4]:
def get_liquidation_period(portfolio):
    check = [i['execution_lag'] + math.ceil(abs(i['quantity']) / i['market_depth']) for i in portfolio]

    return max(check) - 1

In [5]:
H = get_liquidation_period(portfolio)
H

3

#### Риск-сценарии $\mathbf{r_k}$
Они же ценовые пути, price_paths $P_i(\mathbf{r_k})$

Генерация производится непосредственно до реализации методологии CORE, в ней необходимы только ценовые пути

In [6]:
# временнная функция, присобачиваем Q_{i,0} к price_path
def stack_prices(portfolio, price_path):
    def_price = np.array([i['spot'] for i in portfolio]).reshape(len(portfolio), 1)

    return np.hstack((def_price, price_path))

Три сценария

In [ ]:
price_path1 = np.array([[130, 150, 160], # цена 1-го актива в течении времени
                         [130, 150, 160]]) # цена 2-го актива в течении времени
price_path1 = stack_prices(portfolio, price_path1)

price_path2 = np.array([[80, 70, 60], # цена 1-го актива в течении времени
                         [80, 70, 60]]) # цена 2-го актива в течении времени
price_path2 = stack_prices(portfolio, price_path2)

price_path3 = np.array([[110, 60, 80], # цена 1-го актива в течении времени
                        [110, 60, 80]]) # цена 2-го актива в течении времени
price_path3 = stack_prices(portfolio, price_path3)

In [10]:
price_paths = np.array([price_path1,
                        price_path2,
                        price_path3])
price_paths, price_paths.shape

(array([[[100, 130, 150, 160],
         [100, 130, 150, 160]],
 
        [[100,  80,  70,  60],
         [100,  80,  70,  60]],
 
        [[100, 110,  60,  80],
         [100, 110,  60,  80]]]),
 (3, 2, 4))

#### Определение вспомогательной функций: $\psi_i (\mathbf{r_k}, h)$
Показывает потенциальный P&L для инструмента i при времени h в риск-сценарии $\mathbf{r_k}$
По факту из риск-сценария требуется ценовой путь

In [11]:
def portfolio_psi(portfolio, price_paths):
    quantities = np.array([i['quantity'] for i in portfolio]).reshape(len(portfolio), 1)
    ds = np.array([i['DS'] for i in portfolio]).reshape(len(portfolio),1)

    mid = price_paths[:, :, 1:] - ds * price_paths[:, :, :-1]
    return mid * quantities

In [31]:
# если будет класс фреймворка, то пси должно стать отдельным атрибутом
psi = portfolio_psi(portfolio, price_paths)
psi

array([[[ 130,  150,  160],
        [-260, -300, -320]],

       [[  80,   70,   60],
        [-160, -140, -120]],

       [[ 110,   60,   80],
        [-220, -120, -160]]])

#### Оптимизация

<img src="img.png" alt="img" width="500"/>

In [13]:
strats_for_1 = [0, 0, 1]
strats_for_2 = [[2, 0, 0],
                [1, 1, 0],
                [1, 0, 1],
                [0, 1, 1],
                [0, 2, 0],
                [0, 0, 2]]

quantities = np.array([i['quantity'] for i in portfolio]).reshape(len(portfolio), 1)
ds = np.array([i['DS'] for i in portfolio]).reshape(len(portfolio), 1)
abs_quantities = np.abs(quantities)

In [14]:
for j in strats_for_2:
    strategy = np.array([strats_for_1, j])

    step1 = ds * (abs_quantities - np.cumsum(strategy, axis=1)) + strategy
    step2 = psi * step1 / abs_quantities
    step3 = np.sum(step2, axis=1).cumsum(axis=1)
    print(step3)

    first = step3[:,-1] # L(r_k, H)
    second = np.min(step3, axis=1) # \min_{h} L(r_k, h)
    print(np.min(first + second))

[[-260. -260. -100.]
 [-160. -160. -100.]
 [-220. -220. -140.]]
-360.0
[[-130. -280. -120.]
 [ -80. -150.  -90.]
 [-110. -170.  -90.]]
-400.0
[[-130. -130. -130.]
 [ -80.  -80.  -80.]
 [-110. -110. -110.]]
-260.0
[[   0. -150. -150.]
 [   0.  -70.  -70.]
 [   0.  -60.  -60.]]
-300.0
[[   0. -300. -140.]
 [   0. -140.  -80.]
 [   0. -120.  -40.]]
-440.0
[[   0.    0. -160.]
 [   0.    0.  -60.]
 [   0.    0.  -80.]]
-320.0


Целевая функция

In [15]:
def objective_func(portfolio, psi, strategy, type="WL"):
    """
    Objective function defined:

    1) for permanent and worst transient loss in the worst case scenario
        L_{WL} = \min_{r_k} [L(r_k, H) + \min_{h} L(r_k, h)]

    2) for permanent loss in the worst case scenario
        L_{PL} = \min_{r_k} L(r_k, H)

    3) for transient loss in the worst case scenario
        L_{TL} = \min_{r_k} \min_{h} L(r_k, h)


    :param portfolio: portfolio of assets as list of dicts
    :param psi: numpy array of portfolio psi
    :param strategy: dummy strategy
    :param type: type of objective {"WL", "PL", "TL"}

    :return: the value of objective, in other words, c-value
    """
    quantities = np.array([i['quantity'] for i in portfolio]).reshape(len(portfolio), 1)
    ds = np.array([i['DS'] for i in portfolio]).reshape(len(portfolio), 1)
    abs_quantities = np.abs(quantities)

    mid = ds * (abs_quantities - np.cumsum(strategy, axis=1)) + strategy
    mid2 = psi * mid / abs_quantities
    mid3 = np.sum(mid2, axis=1).cumsum(axis=1)

    first = mid3[:,-1] # L(r_k, H)
    second = np.min(mid3, axis=1) # \min_{h} L(r_k, h)

    if type.upper() == "PL":
        return np.min(first)

    if type.upper() == "TL":
        return np.min(second)

    return np.min(first + second)

In [16]:
for j in strats_for_2:
    strategy = np.array([strats_for_1, j])
    print(objective_func(portfolio, psi, strategy))

-360.0
-400.0
-260.0
-300.0
-440.0
-320.0


In [17]:
h

NameError: name 'h' is not defined

#### Пробуем GEKKO

<img src="img2.png" alt="img2" width="500"/>

In [19]:
from gekko import GEKKO

In [20]:
m = GEKKO()

In [33]:
# задаём переменную
# 2 условие Q_{i,h} >= 0
# 3 условие Q_{i,h} <= Q_{i,0}
strategy = [[m.Var(lb=0, ub=abs(i['quantity']), integer=True) for h in range(H)] for i in portfolio]
np.array(strategy)

array([[0, 0, 0],
       [0, 0, 0]], dtype=object)

In [28]:
# 1 условие \sum_{h=1}^{H} Q_{i,h} = Q_{i,0}
for i in range(len(portfolio)):
    temp = 0
    for h in range(H):
        temp += strategy[i][h]

    m.Equation(portfolio[i]['quantity'] == temp)
    print(m.Equation(portfolio[i]['quantity'] == temp))

(((0+v1)+v2)+v3)=1
(((0+v4)+v5)+v6)=-2


In [ ]:
# 5 условие
for i in range(len(portfolio)):
    temp = 0
    for h in range(H):
        temp += strategy[i][h]

    m.Equation(portfolio[i]['quantity'] == temp)
    print(m.Equation(portfolio[i]['quantity'] == temp))